In [3]:
f=open('big_tech_hearing_text.txt', 'r', encoding='utf-8')
txt = f.read()
f.close()

txt[:100]

'Rep. Cicilline: (00:00)\n… Facebook and Google. Before we begin, I’d like to remind members that we h'

In [4]:
import pandas as pd

txt = txt.split('\n')

In [5]:
txt = pd.DataFrame(txt)
txt.head()

,0
0,Rep. Cicilline: (00:00)
1,"… Facebook and Google. Before we begin, I’d li..."
2,Rep. Cicilline: (00:23)
3,I would also remind all members that guidance ...
4,Rep. Cicilline: (00:42)


In [6]:
txt['Speech'] = ''
print(type(txt))

for index, row in txt.iterrows():
    #print(row.name)
    if row.name % 2 == 1:
        #print('speech: {}'.format(row))
        txt.iloc[row.name-1].Speech = row[0]
        #print(txt.iloc[row.name-1])

<class 'pandas.core.frame.DataFrame'>


In [7]:
import numpy as np

txt.drop(txt[txt.index%2 == 1].index,inplace=True)
txt.reset_index(drop='True',inplace=True)
txt.rename(columns={0:'Speaker'}, inplace=True)
#txt = txt[txt['Speech'].str.contains('%')]
#txt['Percent Character'] = txt['Speech'].str.findall('%')
txt.head()

,Speaker,Speech
0,Rep. Cicilline: (00:00),"… Facebook and Google. Before we begin, I’d li..."
1,Rep. Cicilline: (00:23),I would also remind all members that guidance ...
2,Rep. Cicilline: (00:42),"More than a year ago, this subcommittee launch..."
3,Rep. Cicilline: (01:06),"Since then, we’ve received millions of pages o..."
4,Rep. Cicilline: (01:36),This investigation has been bipartisan from th...


In [20]:
def get_percentage(row):
    pctg = []
    for word in row['Speech'].split():
        if word.find('%') != -1:
            pctg.append(float(word.strip('?').strip(',').strip('.').strip('%').replace(',','').split('-')[-1]))
    return pctg
    
txt['Percentage'] = txt.apply(get_percentage, axis=1)
txt.head(10)

,Speaker,Speech,Percentage,Is CEO,Words
0,Rep. Cicilline,"… Facebook and Google. Before we begin, I’d li...",[],False,79
1,Rep. Cicilline,I would also remind all members that guidance ...,[],False,59
2,Rep. Cicilline,"More than a year ago, this subcommittee launch...",[],False,71
3,Rep. Cicilline,"Since then, we’ve received millions of pages o...",[],False,87
4,Rep. Cicilline,This investigation has been bipartisan from th...,[],False,94
5,Rep. Cicilline,The purpose of today’s hearing is to examine t...,[70.0],False,75
6,Rep. Cicilline,Apple is a dominant provider of smartphones wi...,[],False,35
7,Rep. Cicilline,Facebook is the world’s largest provider of so...,[],False,34
8,Rep. Cicilline,"Lastly, Google is the world’s largest online s...",[90.0],False,37
9,Rep. Cicilline,"Prior to the COVID-19 pandemic, these corporat...",[],False,69


In [18]:
def is_CEO(row):
    return ((row['Speaker'][:7] == 'Jeff Be') 
            or (row['Speaker'][:7] == 'Mr. Pic') 
            or (row['Speaker'][:7] == 'Mr. Zuc') 
            or(row['Speaker'][:7] == 'Mark Zu')
            or(row['Speaker'][:7] == 'Tim Coo'))
txt['Is CEO'] = txt.apply(is_CEO, axis=1)        

def speaker_name(row):
    name = row['Speaker'].split()[0:2]
    name = ' '.join(name).strip(':')
    return name  
txt['Speaker'] = txt.apply(speaker_name, axis=1)  

txt.loc[80,'Speaker'] = 'Mr. Zuckerberg'

In [10]:
txt['Words'] = txt['Speech'].apply(lambda x: len(x.split()))
txt.head()

,Speaker,Speech,Percentage,Is CEO,Words
0,Rep. Cicilline,"… Facebook and Google. Before we begin, I’d li...",[],False,79
1,Rep. Cicilline,I would also remind all members that guidance ...,[],False,59
2,Rep. Cicilline,"More than a year ago, this subcommittee launch...",[],False,71
3,Rep. Cicilline,"Since then, we’ve received millions of pages o...",[],False,87
4,Rep. Cicilline,This investigation has been bipartisan from th...,[],False,94


In [11]:
import matplotlib.pyplot as plt
%matplotlib widget

#fig = plt.figure()
fig = plt.figure(figsize=(20, 5), facecolor='w', edgecolor='k')

plt.bar(txt[txt['Is CEO'] == True].index,txt[txt['Is CEO'] == True]['Words'], Color='Blue', width=.8,linewidth=0, alpha=.6)
plt.bar(txt[txt['Is CEO'] == False].index,txt[txt['Is CEO'] == False]['Words'], Color='Gold', width=.8,linewidth=0)
plt.ylabel('Number of Words Said')
plt.xlabel('Transcript Entry')
plt.title('Number of Words Said by Big Tech CEOs in Antitrust Hearing (July 29, 2020)', fontsize=16)
plt.legend(['Tech CEOs                                       (Total Words = 15,731)','House Representatives and Other   (Total Words = 31,154)'])


ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.axes.xaxis.set_visible(False)
ax.tick_params(axis='y', labelsize=6)

#list source
plt.text(890, -10, 'Source: Official Hearing Transcript', fontsize=5)

sp_list = []
def onclick(event):
    for i, a in enumerate(sp_list):
        a.remove()
    sp_list[:] = []
    sp = plt.annotate(txt['Speaker'][int(event.xdata)],(event.xdata,event.ydata))
    sp_list.append(sp)
    
_ = plt.gcf().canvas.mpl_connect('motion_notify_event',onclick)

#plt.savefig('hearing_words.pdf', bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
pcttxt = txt[txt['Speech'].str.contains('%')]
pct_df = pd.DataFrame(columns=['Line','Percentage','Speaker', 'Is CEO'])
for index,row in pcttxt.iterrows():
    #print(row)
    for pct in row['Percentage']:
        pct_df = pct_df.append(other={'Line':row.name,'Percentage':pct,'Speaker':row['Speaker'], 'Is CEO':row['Is CEO']}, ignore_index=True)

pct_df

,Line,Percentage,Speaker,Is CEO
0,5,70.0,Rep. Cicilline,False
1,8,90.0,Rep. Cicilline,False
2,47,99.0,Rep. Jordan,False
3,85,80.0,Jeff Bezos,True
4,85,1.0,Jeff Bezos,True
...,...,...,...,...
65,845,30.0,Ms. Scanlon,False
66,846,30.0,Jeff Bezos,True
67,846,60.0,Jeff Bezos,True
68,847,11.0,Ms. Scanlon,False


In [13]:
%matplotlib widget

plt.figure()
#plt.scatter(pct_df[pct_df['Is CEO'] == True]['Line'],pct_df[pct_df['Is CEO'] == True]['Percentage'], Color='Blue')
#plt.scatter(pct_df[pct_df['Is CEO'] == False]['Line'],pct_df[pct_df['Is CEO'] == False]['Percentage'], Color='Gold')

ax=plt.gca()

def plot_pct(row):
    if row['Is CEO'] == True:
        plt.plot(int(row.name),int(row['Percentage']),marker='o',color='Blue', markersize= 7+row['Percentage']/20)
        ax.annotate(xy=(int(row.name),int(row['Percentage'])), s=int(row['Percentage']), fontsize=6, ha='center',color='w')
    else:
        plt.plot(int(row.name),int(row['Percentage']),marker='o',color='Gold', markersize=7+row['Percentage']/20)
    return row
pct_df.apply(plot_pct,axis=1)

plt.ylabel('Percentage Quoted (%)')
#plt.legend(['Tech CEOs', 'House Representatives and Other'], loc=1)

plt.ylim(top=130, bottom=-5)

plt.title('Percentages quoted during hearing')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Percentages quoted during hearing')

In [14]:
#pcttxt = txt.groupby(by='Speaker')['Percentage'].sum()

In [15]:
pctpt = pd.pivot_table(txt, index=['Is CEO','Speaker'], aggfunc=[sum], values=['Percentage'])

pctpt.columns = pctpt.columns.droplevel(0)
pctpt['Median'] = pctpt['Percentage'].apply(lambda x: np.median(x))
pctpt['Mean'] = pctpt['Percentage'].apply(lambda x: np.mean(x))
pctpt['STD'] = pctpt['Percentage'].apply(lambda x: np.std(x))

pctpt

C:\Users\alfis\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\alfis\anaconda3\lib\site-packages\numpy\core\_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)


Percentage  \
Is CEO Speaker                                                                 
False  Book Seller                                                        []   
       Greg Steube                                                        []   
       Kelly Armstrong                                       [1500.0, 500.0]   
       Mr. Buck                                                       [43.0]   
       Mr. Cicilline                                                      []   
       Mr. Gates                                                      [10.0]   
       Mr. Johnson                                              [30.0, 50.0]   
       Mr. Jordan                                                         []   
       Mr. Neguse                               [95.0, 10.0, 1.0, 4.0, 40.0]   
       Mr. Raskin                       [60.0, 30.0, 15.0, 30.0, 25.0, 30.0]   
       Ms. McBath                                                         []   
       Ms. Scanlon                                  [19.0, 30.0, 11.0, 21.0]   
       Pramila Jayapal                                          [60.0, 90.0]   
       Rep. Cicilline             [70.0, 90.0, 85.0, 80.0, 63.0, 75.0, 37.0]   
       Rep. Jordan                                                    [99.0]   
       Rep. Nadler                                                    [30.0]   
       Rep. Sensenbrenner                                                 []   
       Speaker 1                                                          []   
       Speaker 2                                                          []   
       Speaker 3                                                          []   
       Speaker 4                                                          []   
       Speaker 5                                                          []   
       Sudar Pichai                                                       []   
       Val Demings                                       [80.0, 80.0, 100.0]   
True   Jeff Bezos             [80.0, 1.0, 4.0, 60.0, 30.0, 40.0, 30.0, 60.0]   
       Mr. Pichai             [40.0, 55.0, 70.0, 40.0, 1.4, 1.0, 40.0, 69.0]   
       Mr. Zuckerberg                               [89.0, 89.0, 99.0, 89.0]   
       Tim Cook            [99.0, 100.0, 84.0, 16.0, 15.0, 70.0, 30.0, 84...   

                           Median         Mean         STD  
Is CEO Speaker                                              
False  Book Seller            NaN          NaN         NaN  
       Greg Steube            NaN          NaN         NaN  
       Kelly Armstrong     1000.0  1000.000000  500.000000  
       Mr. Buck              43.0    43.000000    0.000000  
       Mr. Cicilline          NaN          NaN         NaN  
       Mr. Gates             10.0    10.000000    0.000000  
       Mr. Johnson           40.0    40.000000   10.000000  
       Mr. Jordan             NaN          NaN         NaN  
       Mr. Neguse            10.0    30.000000   35.332704  
       Mr. Raskin            30.0    31.666667   13.743685  
       Ms. McBath             NaN          NaN         NaN  
       Ms. Scanlon           20.0    20.250000    6.759253  
       Pramila Jayapal       75.0    75.000000   15.000000  
       Rep. Cicilline        75.0    71.428571   16.360729  
       Rep. Jordan           99.0    99.000000    0.000000  
       Rep. Nadler           30.0    30.000000    0.000000  
       Rep. Sensenbrenner     NaN          NaN         NaN  
       Speaker 1              NaN          NaN         NaN  
       Speaker 2              NaN          NaN         NaN  
       Speaker 3              NaN          NaN         NaN  
       Speaker 4              NaN          NaN         NaN  
       Speaker 5              NaN          NaN         NaN  
       Sudar Pichai           NaN          NaN         NaN  
       Val Demings           80.0    86.666667    9.428090  
True   Jeff Bezos            35.0    38.125000   25.953986  
       Mr. Pichai 

In [17]:
pt_speakers = pd.pivot_table(txt, index=['Is CEO', 'Speaker'], aggfunc=[np.mean,np.std,np.sum], values=['Percentage'])
pt_isceo = pd.pivot_table(txt, index=['Is CEO'], aggfunc=[np.mean,np.std,np.sum], values=['Percentage', 'Words'])

pt_speakers.sort_values(by=('mean','Words'),ascending=False)

DataError: No numeric types to aggregate